In [80]:
import tkinter as tk
from tkinter import filedialog, simpledialog, messagebox, Scrollbar, OptionMenu, StringVar
from PIL import Image, ImageTk, ImageDraw, ImageFont, ImageSequence
from fpdf import FPDF
import pydicom
import os
import pickle
import numpy as np
from ultralytics import YOLO
import torch
from torchvision import transforms
from torchvision.models.segmentation import deeplabv3_resnet50
from datetime import datetime
import threading
import ctypes

class MedImagePro:
    def __init__(self, root):
        self.root = root
        self.root.title("MedImagePro")
        
        icon_path = "MedImagePro.ico"
        self.root.iconbitmap(icon_path)

        # Set the taskbar icon on Windows using ctypes
        if os.name == 'nt':  # Check if the operating system is Windows
            ctypes.windll.shell32.SetCurrentProcessExplicitAppUserModelID("myappid")
            self.root.iconbitmap(icon_path)

        # Load models
        self.model = YOLO('yolov8l.pt')
        self.deeplab_model = deeplabv3_resnet50(pretrained=True).eval()

        # Operation box at the top
        self.operation_frame = tk.Frame(root)
        self.operation_frame.pack(side=tk.TOP, fill=tk.X)

        self.load_button = tk.Button(self.operation_frame, text="Load Image", command=self.load_image)
        self.load_button.pack(side=tk.LEFT, padx=5, pady=5)

        # Model selection dropdown
        self.model_var = StringVar(self.root)
        self.model_var.set("YOLOv8")
        self.model_menu = OptionMenu(self.operation_frame, self.model_var, "YOLOv8", "DeepLabV3")
        self.model_menu.pack(side=tk.LEFT, padx=5, pady=5)

        self.process_button = tk.Button(self.operation_frame, text="Apply Filter", command=self.apply_filter)
        self.process_button.pack(side=tk.LEFT, padx=5, pady=5)

        self.undo_button = tk.Button(self.operation_frame, text="Undo", command=self.undo)
        self.undo_button.pack(side=tk.LEFT, padx=5, pady=5)

        self.restore_button = tk.Button(self.operation_frame, text="Restore", command=self.restore)
        self.restore_button.pack(side=tk.LEFT, padx=5, pady=5)

        self.export_button = tk.Button(self.operation_frame, text="Export as PDF", command=self.export_as_pdf)
        self.export_button.pack(side=tk.LEFT, padx=5, pady=5)

        self.export_dicom_button = tk.Button(self.operation_frame, text="Export as DICOM", command=self.export_modified_dicom)
        self.export_dicom_button.pack(side=tk.LEFT, padx=5, pady=5)

        self.save_button = tk.Button(self.operation_frame, text="Save Project", command=self.save_project)
        self.save_button.pack(side=tk.LEFT, padx=5, pady=5)

        self.load_project_button = tk.Button(self.operation_frame, text="Load Project", command=self.load_project)
        self.load_project_button.pack(side=tk.LEFT, padx=5, pady=5)

        # Main frame for images and zoom controls
        self.main_frame = tk.Frame(root)
        self.main_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        # Toggle button for DICOM details
        self.details_toggle_button = tk.Button(self.main_frame, text="⮞", command=self.toggle_details_visibility)
        self.details_toggle_button.pack(side=tk.LEFT, padx=5, pady=5)

        # Details Frame on the left, initially hidden
        self.details_frame = tk.Frame(self.main_frame, width=200)
        self.details_frame.pack(side=tk.LEFT, fill=tk.Y)
        self.details_frame.pack_forget()  # Initially hidden

        self.details_label = tk.Label(self.details_frame, text="DICOM Details")
        self.details_label.pack(side=tk.TOP, anchor="w", padx=5, pady=5)

        self.details_text = tk.Text(self.details_frame, height=15, width=30)
        self.details_text.pack(side=tk.TOP, fill=tk.Y, padx=5, pady=5, expand=True)

        self.details_frame_visible = False

        # Images Frame
        self.images_frame = tk.Frame(self.main_frame)
        self.images_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        try:
            # Load and resize the images
            logo1_image = Image.open("VIML.png").resize((80, 80), Image.LANCZOS)
            self.logo1 = ImageTk.PhotoImage(logo1_image)
            
            logo2_image = Image.open("IITMANDI.png").resize((100, 100), Image.LANCZOS)
            self.logo2 = ImageTk.PhotoImage(logo2_image)
            
            # Create a frame to hold the logos
            self.logo_frame = tk.Frame(root)
            self.logo_frame.pack(side=tk.BOTTOM, anchor=tk.SE, pady=5)
            
            # Add the logos to the frame side by side
            self.logo1_label = tk.Label(self.logo_frame, image=self.logo1)
            self.logo1_label.pack(side=tk.LEFT, padx=5)
            
            self.logo2_label = tk.Label(self.logo_frame, image=self.logo2)
            self.logo2_label.pack(side=tk.LEFT, padx=5)
        
        except Exception as e:
            print(f"Error loading logo images: {e}")

        # Original Image Frame
        self.original_frame = tk.Frame(self.images_frame, width=100, height=100, bg="white")
        self.original_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)

        self.zoom_scale = tk.Scale(self.images_frame, from_=10, to=1, orient=tk.VERTICAL, resolution=0.1, command=self.zoom_images)
        self.zoom_scale.pack(side=tk.LEFT, fill=tk.X, padx=5, pady=5)
        self.zoom_scale.set(1)

        self.original_canvas = tk.Canvas(self.original_frame, bg="grey")
        self.original_canvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        self.original_bbox_button = tk.Button(self.original_frame, text="Hide BBoxes", command=self.toggle_original_bboxes)
        self.original_bbox_button.pack(side=tk.BOTTOM, padx=5, pady=5)
        self.original_bboxes_visible = True

        # Processed Image Frame
        self.processed_frame = tk.Frame(self.images_frame, width=100, height=100, bg="white")
        self.processed_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)

        self.processed_canvas = tk.Canvas(self.processed_frame, bg="grey", cursor="cross")
        self.processed_canvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        self.processed_bbox_button = tk.Button(self.processed_frame, text="Hide BBoxes", command=self.toggle_processed_bboxes)
        self.processed_bbox_button.pack(side=tk.BOTTOM, padx=5, pady=5)
        self.processed_bboxes_visible = True

        try:
            history_icon_image = Image.open("history.png").resize((20, 20), Image.LANCZOS)
            self.history_icon = ImageTk.PhotoImage(history_icon_image)
            self.history_button = tk.Button(self.main_frame, image=self.history_icon, command=self.toggle_history_log)
            self.history_button.image = self.history_icon  # Keep reference to avoid GC
            self.history_button.pack(side=tk.RIGHT, padx=5, pady=0)
        except Exception as e:
            print(f"Error loading history icon: {e}")
            self.history_button = tk.Button(self.main_frame, text="History", command=self.toggle_history_log)
            self.history_button.pack(side=tk.LEFT, padx=5, pady=5)

        # History Sidebar Frame, initially hidden
        self.history_frame = tk.Frame(self.main_frame, width=200)
        self.history_frame.pack(side=tk.RIGHT, fill=tk.Y)
        self.history_frame.pack_forget()  # Initially hidden

        self.history_label = tk.Label(self.history_frame, text="History")
        self.history_label.pack(side=tk.TOP, anchor="w", padx=5, pady=5)

        self.history_listbox = tk.Listbox(self.history_frame)
        self.history_listbox.pack(side=tk.TOP, fill=tk.Y, padx=5, pady=5, expand=True)
        self.history_listbox.bind('<Double-1>', self.revert_to_history_item)

        self.history_frame_visible = False

        # Comment Frame
        self.comment_frame = tk.Frame(root, height=100, bg="white")
        self.comment_frame.pack(side=tk.LEFT, fill=tk.BOTH, padx=60, pady=10)

        self.comment_scrollbar = Scrollbar(self.comment_frame)
        self.comment_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.comment_text = tk.Text(self.comment_frame, wrap=tk.WORD, yscrollcommand=self.comment_scrollbar.set, height=5)
        self.comment_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.comment_scrollbar.config(command=self.comment_text.yview)
        
        self.processed_canvas.bind("<Button-1>", self.canvas_click)
        self.processed_canvas.bind("<Double-1>", self.canvas_double_click)
        self.processed_canvas.bind("<ButtonRelease-1>", self.canvas_release)
        self.processed_canvas.bind("<B1-Motion>", self.move_bounding_box)
        self.root.bind("<Delete>", self.delete_bbox_keypress)
        self.original_canvas.bind("<ButtonPress-1>", self.pan_start)
        self.original_canvas.bind("<B1-Motion>", self.pan_move)
        self.detections = []
        self.bbox_handles = {}
        self.bbox_labels = {}
        self.detection_comments = {}
        self.selected_bbox = None
        self.move_start = None
        self.resizing = False
        self.resize_handle = None

        self.original_image = None
        self.processed_image = None
        self.dicom_image = None
        self.image_scale = 1
        self.original_zoom_factor = 1
        self.processed_zoom_factor = 1

        self.history = []
        self.initial_state = None
        
        # Load the loading GIF
        loading_image = Image.open('loading.gif')
        self.loading_gif = []

        for frame in ImageSequence.Iterator(loading_image):
    # Convert frame to RGBA mode
            frame = frame.convert("RGBA")
    
    # Ensure transparency by iterating through pixels
            data = frame.getdata()
            new_data = []
            for item in data:
        # Set white (255, 255, 255) pixels to transparent
                if item[0] == 255 and item[1] == 255 and item[2] == 255:
                    new_data.append((255, 255, 255, 0))
                else:
                    new_data.append(item)
    
    # Create a new image with transparency applied
            frame.putdata(new_data)
    
    # Resize the frame
            resized_frame = frame.resize((100, 100), Image.LANCZOS)
    
    # Append PhotoImage to self.loading_gif
            self.loading_gif.append(ImageTk.PhotoImage(resized_frame))

    def load_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            if file_path.lower().endswith('.dcm'):
                self.dicom_image = pydicom.dcmread(file_path)
                pixel_array = self.dicom_image.pixel_array
                pixel_array = self.normalize_pixel_array(pixel_array)
                image = Image.fromarray(pixel_array).convert('L')
                self.details_text.insert(tk.END, str(self.dicom_image))
            else:
                image = Image.open(file_path)

            self.original_image = image
            self.processed_image = image.copy()
            self.display_image(self.original_canvas, self.original_image)
            self.display_image(self.processed_canvas, self.processed_image)
            self.save_initial_state()
            self.history = []  # Reset history on new image load
            self.log_history('Image loaded')

    def normalize_pixel_array(self, pixel_array):
        if np.issubdtype(pixel_array.dtype, np.integer):
            max_val = np.max(pixel_array)
            if max_val > 0:
                pixel_array = (pixel_array / max_val) * 255.0
            pixel_array = pixel_array.astype(np.uint8)
        elif np.issubdtype(pixel_array.dtype, np.float):
            pixel_array = (pixel_array - np.min(pixel_array)) / (np.max(pixel_array) - np.min(pixel_array))
            pixel_array = (pixel_array * 255).astype(np.uint8)
        return pixel_array
    
    def show_loading_animation(self):
        self.loading_label = tk.Label(self.main_frame,)
        self.loading_label.place(relx=0.5, rely=0.5, anchor=tk.CENTER)
        self.update_loading_frame(0)

    def update_loading_frame(self, frame):
        if self.loading_label:
            self.loading_label.configure(image=self.loading_gif[frame])
            frame = (frame + 1) % len(self.loading_gif)
            self.root.after(100, self.update_loading_frame, frame)

    def hide_loading_animation(self):
        if self.loading_label:
            self.loading_label.destroy()
            self.loading_label = None

    def apply_filter(self):
        if self.processed_image:
            self.show_loading_animation()
            threading.Thread(target=self.apply_filter_in_background).start()

    def apply_filter_in_background(self):
        # Simulating filter application with sleep
        import time
        time.sleep(3)  # Simulate long processing time
        
        self.log_history('Before filter')
        if not self.original_image:
            messagebox.showerror("Error", "Load an image first.")
            return

        model_name = self.model_var.get()
        if model_name == "YOLOv8":
            self.apply_yolo_filter()
        elif model_name == "DeepLabV3":
            self.apply_deeplab_filter()

        self.save_initial_state()
        # Here you would apply your actual filter logic
        self.hide_loading_animation()
        

    def apply_yolo_filter(self):
        if not self.original_image:
            messagebox.showerror("Error", "Load an image first.")
            return

        image_np = np.array(self.original_image.convert("RGB"))
        results = self.model(image_np)

        self.detections.clear()
        for i, result in enumerate(results):
            for j, bbox in enumerate(result.boxes):
                x1, y1, x2, y2 = bbox.xyxy[0].int().tolist()
                class_name = result.names[bbox.cls[0].item()]
                confidence = bbox.conf[0].item()
                detection_id = len(self.detections)
                self.detections.append({
                    'id': detection_id,
                    'bbox': (x1, y1, x2, y2),
                    'class': class_name,
                    'confidence': confidence,
                    'comment': ''
                })
        self.save_initial_state()
        self.display_image(self.original_canvas, self.original_image, self.detections)
        self.display_image(self.processed_canvas, self.processed_image, self.detections)
        self.log_history('YOLOV8 filter applied')

    def apply_deeplab_filter(self):
        preprocess = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

        input_tensor = preprocess(self.original_image.convert("RGB"))
        input_batch = input_tensor.unsqueeze(0)

        with torch.no_grad():
            output = self.deeplab_model(input_batch)['out'][0]
        output_predictions = output.argmax(0).byte().cpu().numpy()

        mask = Image.fromarray(output_predictions)
        mask = mask.resize(self.original_image.size, Image.NEAREST)
        mask = mask.convert("L")

        self.processed_image = Image.composite(self.original_image.convert("RGBA"), mask.convert("RGBA"), mask)
        self.display_image(self.original_canvas, self.original_image, self.detections)
        self.display_image(self.processed_canvas, self.processed_image)
        self.log_history('DeepLabV3 filter applied')

    def display_image(self, canvas, image, detections=[]):
        image = image.resize((int(image.width * self.image_scale), int(image.height * self.image_scale)), Image.LANCZOS)
        canvas.image = ImageTk.PhotoImage(image)
        canvas.create_image(0, 0, anchor=tk.NW, image=canvas.image)
        canvas.config(scrollregion=canvas.bbox(tk.ALL))
        canvas.delete("bbox")

        if (self.original_bboxes_visible and canvas == self.original_canvas) or \
           (self.processed_bboxes_visible and canvas == self.processed_canvas):
            for detection in detections:
                x1, y1, x2, y2 = detection['bbox']
                self.draw_bounding_boxes()

    def draw_bounding_boxes(self):
        self.processed_image = self.original_image.copy()
        draw = ImageDraw.Draw(self.processed_image)
        font_path = "arial.ttf"
        font = ImageFont.truetype(font_path, 15) if os.path.exists(font_path) else ImageFont.load_default()

        for detection in self.detections:
            x1, y1, x2, y2 = detection['bbox']
            class_name = detection['class']
            confidence = detection['confidence']
            detection_id = detection['id']

            bbox_color = "red"
            if self.selected_bbox is not None and detection_id == self.selected_bbox:
                bbox_color = "green"

            draw.rectangle([x1, y1, x2, y2], outline=bbox_color, width=2)
            label = f"{detection_id}: {class_name} {confidence:.2f}"

            text_bbox = draw.textbbox((0, 0), label, font=font)
            text_width = text_bbox[2] - text_bbox[0]
            text_height = text_bbox[3] - text_bbox[1]
            text_bg = Image.new("RGB", (text_width + 6, text_height + 6), bbox_color)
            text_bg_draw = ImageDraw.Draw(text_bg)
            text_bg_draw.text((3, 3), label, fill="white", font=font)
            self.processed_image.paste(text_bg, (x1, y1 - text_height - 6))

        # Display comments
        comments = '\n'.join([f"ID {d['id']}: {d['comment']}" for d in self.detections if d['comment']])
        self.comment_text.delete("1.0", tk.END)
        self.comment_text.insert(tk.END, comments)


    def toggle_original_bboxes(self):
        self.original_bboxes_visible = not self.original_bboxes_visible
        self.display_image(self.original_canvas, self.original_image, self.detections)
        self.original_bbox_button.config(text="Show BBoxes" if not self.original_bboxes_visible else "Hide BBoxes")

    def toggle_processed_bboxes(self):
        self.processed_bboxes_visible = not self.processed_bboxes_visible
        self.display_image(self.processed_canvas, self.processed_image, self.detections)
        self.processed_bbox_button.config(text="Show BBoxes" if not self.processed_bboxes_visible else "Hide BBoxes")

    def zoom_images(self, value):
        zoom_factor = float(value)
        self.image_scale = zoom_factor
        self.update_zoom(self.original_canvas, self.original_image, zoom_factor)
        self.update_zoom(self.processed_canvas, self.processed_image, zoom_factor)
        
    def update_zoom(self, canvas, image, zoom_factor):
        if image:
            width, height = image.size
            new_size = (int(width * zoom_factor), int(height * zoom_factor))
            resized_image = image.resize(new_size, Image.LANCZOS)
        
            canvas.image = ImageTk.PhotoImage(resized_image)
            canvas.create_image(0, 0, anchor=tk.NW, image=canvas.image)
            canvas.config(scrollregion=canvas.bbox(tk.ALL))
            self.sync_canvases()
            
    def pan_start(self, event):
        self.original_canvas.scan_mark(event.x, event.y)

    def pan_move(self, event):
        self.original_canvas.scan_dragto(event.x, event.y, gain=1)
        self.sync_canvases()

    def sync_canvases(self):
        # Get the scroll region of the original canvas
        x0, x1 = self.original_canvas.xview()
        y0, y1 = self.original_canvas.yview()
        
        # Set the processed canvas view to match the original canvas
        self.processed_canvas.xview_moveto(x0)
        self.processed_canvas.yview_moveto(y0)

    def save_initial_state(self):
        self.initial_state = {
            'original_image': self.original_image,
            'processed_image': self.processed_image,
            'detections': [d.copy() for d in self.detections]
        }

    def restore(self):
        if self.initial_state:
            self.original_image = self.initial_state['original_image']
            self.processed_image = self.initial_state['processed_image']
            self.detections = [d.copy() for d in self.initial_state['detections']]
            self.display_image(self.original_canvas, self.original_image)
            self.display_image(self.processed_canvas, self.processed_image)
            self.update_bounding_boxes()
            self.update_comment_section()

    def log_history(self):
        self.history.append({
            'original_image': self.original_image.copy(),
            'processed_image': self.processed_image.copy(),
            'detections': [d.copy() for d in self.detections]
        })
    def undo(self):
        if self.history:
            last_state = self.history.pop()
            self.original_image = last_state['original_image']
            self.processed_image = last_state['processed_image']
            self.detections = last_state['detections']
            self.display_image(self.original_canvas, self.original_image)
            self.display_image(self.processed_canvas, self.processed_image)
            self.update_bounding_boxes()
            self.update_comment_section()

    def save_project(self):
        file_path = filedialog.asksaveasfilename(defaultextension=".mip", filetypes=[("MedImagePro Projects", "*.mip")])
        if file_path:
            project_data = {
                'original_image': self.original_image,
                'processed_image': self.processed_image,
                'dicom_image': self.dicom_image,
                'detections': self.detections,
                'comments': self.detection_comments
            }
            with open(file_path, 'wb') as file:
                pickle.dump(project_data, file)

    def load_project(self):
        file_path = filedialog.askopenfilename(defaultextension=".mip", filetypes=[("MedImagePro Projects", "*.mip")])
        if file_path:
            with open(file_path, 'rb') as file:
                project_data = pickle.load(file)
                self.original_image = project_data['original_image']
                self.processed_image = project_data['processed_image']
                self.dicom_image = project_data['dicom_image']
                self.detections = project_data['detections']
                self.detection_comments = project_data['comments']
                self.display_image(self.original_canvas, self.original_image)
                self.display_image(self.processed_canvas, self.processed_image)
    def canvas_click(self, event):
        self.selected_bbox = self.find_bbox(event.x, event.y)
        if self.selected_bbox is not None:
            self.move_start = (event.x, event.y)
            self.resizing = self.is_resize_handle(event.x, event.y)
            self.draw_bounding_boxes()
            self.display_image(self.processed_canvas, self.processed_image)

    def canvas_double_click(self, event):
        bbox_id = self.find_bbox(event.x, event.y)
        if bbox_id is not None:
            comment = simpledialog.askstring("Add Comment", f"Enter your comment for bbox ID {bbox_id}:")
            if comment:
                self.detection_comments[bbox_id] = comment
                self.detections[bbox_id]['comment'] = comment
                self.draw_bounding_boxes()
                self.log_history()
                self.display_image(self.processed_canvas, self.processed_image)

    def canvas_release(self, event):
        self.selected_bbox = None
        self.move_start = None
        self.resizing = False
        self.resize_handle = None
        self.draw_bounding_boxes()
        self.log_history()
        self.display_image(self.processed_canvas, self.processed_image)

    def move_bounding_box(self, event):
        if self.selected_bbox is not None:
            dx = event.x - self.move_start[0]
            dy = event.y - self.move_start[1]
            self.move_start = (event.x, event.y)
            bbox = list(self.detections[self.selected_bbox]['bbox'])
            if self.resizing:
                self.resize_bbox(bbox, dx, dy)
            else:
                bbox[0] += dx
                bbox[1] += dy
                bbox[2] += dx
                bbox[3] += dy
            self.detections[self.selected_bbox]['bbox'] = tuple(bbox)
            self.draw_bounding_boxes()
            self.display_image(self.processed_canvas, self.processed_image)

    def is_resize_handle(self, x, y):
        if self.selected_bbox is None:
            return False
        bbox = self.detections[self.selected_bbox]['bbox']
        handle_size = 10
        x1, y1, x2, y2 = bbox
        if abs(x - x2) <= handle_size and abs(y - y2) <= handle_size:
            self.resize_handle = 'bottom_right'
            return True
        return False

    def resize_bbox(self, bbox, dx, dy):
        if self.resize_handle == 'bottom_right':
            bbox[2] += dx
            bbox[3] += dy

    def delete_bbox_keypress(self, event):
        if self.selected_bbox is not None:
            del self.detections[self.selected_bbox]
            self.selected_bbox = None
            self.draw_bounding_boxes()
            self.display_image(self.processed_canvas, self.processed_image)
            self.log_history('BBOX Deleted')

    def find_bbox(self, x, y):
        for detection in self.detections:
            x1, y1, x2, y2 = detection['bbox']
            if x1 <= x <= x2 and y1 <= y <= y2:
                return detection['id']
        return None

    def toggle_details_visibility(self):
        if self.details_frame_visible:
            self.details_frame.pack_forget()
            self.details_toggle_button.config(text="⮞")
        else:
            self.details_frame.pack(side=tk.LEFT, fill=tk.Y)
            self.details_toggle_button.config(text="⮜")
        self.details_frame_visible = not self.details_frame_visible

    def export_as_pdf(self):
        if self.processed_image is None:
            messagebox.showerror("Error", "No processed image to export.")
            return

        file_path = filedialog.asksaveasfilename(defaultextension=".pdf", filetypes=[("PDF files", "*.pdf")])
        if file_path:
            pdf = FPDF()
            pdf.set_auto_page_break(auto=True, margin=10)
            
            # Define logo images and their dimensions
            logo_viml = "VIML.png"
            logo_iitmandi = "IITMANDI.png"
            logo_width = 20  # Adjust as per your requirement
            logo_height = 20  # Adjust as per your requirement

            # Header with logos
            def add_header():
                pdf.image(logo_viml, x=10, y=8, w=logo_width)
                pdf.image(logo_iitmandi, x=pdf.w - logo_width - 10, y=8, w=logo_width)
                pdf.ln(20)  # Add a vertical space below the logos

            # Add the first page with header and main title
            pdf.add_page()
            add_header()
        
            # Title
            pdf.set_font("Arial", size=16, style='B')
            pdf.cell(200, 10, txt="MedImagePro Report", ln=True, align="C")
            pdf.ln(10)  # Add a larger vertical space

            # File name and date/time
            pdf.set_font("Arial", size=12)
            file_name = os.path.basename(file_path)
            current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            pdf.cell(0, 10, txt=f"File Name: {file_name}", ln=True)
            pdf.cell(0, 10, txt=f"Date and Time: {current_datetime}", ln=True)
            pdf.ln(5)

            # Original and Processed Images Side by Side with Titles
            pdf.set_font("Arial", size=10, style='B')
            pdf.cell(90, 10, txt="Original Image", border=0, ln=0, align="C")
            pdf.cell(90, 10, txt="Processed Image", border=0, ln=1, align="C")
            pdf.ln(5)

            # Save and insert original image
            original_image_path = "original_image_temp.png"
            self.original_image.save(original_image_path)
            pdf.image(original_image_path, x=10, y=pdf.get_y(), w=90)
            os.remove(original_image_path)

            # Save and insert processed image
            processed_image_path = "processed_image_temp.png"
            self.processed_image.save(processed_image_path)
            pdf.image(processed_image_path, x=110, y=pdf.get_y(), w=90)
            os.remove(processed_image_path)

            pdf.ln(95)  # Ensure enough space after images


            case_count = 0

            for idx, detection in enumerate(self.detections):
                if detection['comment']:
                    case_number = case_count + 1
                    if case_count == 1 or case_count % 3 == 1:
                        pdf.add_page()  # Start a new page for every 3 cases
                        add_header()
                        pdf.ln(5)

                    # Case title
                    pdf.set_font("Arial", size=10, style='B')
                    pdf.cell(0, 10, txt=f"Case No : {case_number}", ln=True)
                
                    # ID and comment
                    pdf.set_font("Arial", size=10)
                    pdf.cell(0, 10, txt=f"ID {detection['id']} : {detection['comment']}", ln=True)
                    pdf.ln(5)

                    # Titles for Bbox Images
                    pdf.cell(90, 10, txt="Original Portion", border=0, ln=0, align="C")
                    pdf.cell(90, 10, txt="Original Portion with Bbox", border=0, ln=1, align="C")
                    pdf.ln(5)

                    # Original portion of the bbox
                    bbox = detection['bbox']
                    x1, y1, x2, y2 = bbox
                    temp_img_original_path = f"bbox_{case_number}_original_temp.png"
                    temp_img_original = self.original_image.crop((x1, y1, x2, y2))
                    temp_img_original.save(temp_img_original_path)
                    pdf.image(temp_img_original_path, x=30, y=pdf.get_y(), w=45)
                    os.remove(temp_img_original_path)

                    # Processed portion with only the main bbox
                    extra_x = int((x2 - x1) * 0.05)
                    extra_y = int((y2 - y1) * 0.05)
                    x1_exp = max(0, x1 - extra_x)
                    y1_exp = max(0, y1 - extra_y)
                    x2_exp = min(self.processed_image.width, x2 + extra_x)
                    y2_exp = min(self.processed_image.height, y2 + extra_y)

                    # Create a copy of the original image and draw only the specific bbox with details
                    temp_img_with_bbox = self.original_image.copy()
                    draw = ImageDraw.Draw(temp_img_with_bbox)
                    draw.rectangle([(x1, y1), (x2, y2)], outline="red", width=3)
                    text = f"{detection['id']}:{detection['class']} {detection['confidence']:.2f}"
                    text_bbox = draw.textbbox((x1, y1), text)
                    draw.rectangle(text_bbox, fill="red")
                    draw.text((x1, y1), text, fill="white")
                    temp_img_with_bbox_cropped = temp_img_with_bbox.crop((x1_exp, y1_exp, x2_exp, y2_exp))
                    temp_img_processed_path = f"bbox_{case_number}_processed_temp.png"
                    temp_img_with_bbox_cropped.save(temp_img_processed_path)
                    pdf.image(temp_img_processed_path, x=120, y=pdf.get_y(), w=45)  # Reduced width
                    os.remove(temp_img_processed_path)

                    pdf.ln(95)  # Add vertical space after images

                    case_count += 1

            # Save and close the PDF
            pdf.output(file_path)
            messagebox.showinfo("Export Successful", f"PDF saved to {file_path}")
            self.log_history('PDF Exported')
    
    def update_bounding_boxes(self):
        for bbox in self.bbox_handles.values():
            self.processed_canvas.delete(bbox)
        self.bbox_handles.clear()
        self.bbox_labels.clear()
        for detection in self.detections:
            x1, y1, x2, y2 = detection['bbox']
            bbox = self.processed_canvas.create_rectangle(x1, y1, x2, y2, outline='red')
            label = self.processed_canvas.create_text(x1, y1 - 10, anchor=tk.SW, text=f"ID {detection['id']}")
            self.bbox_handles[detection['id']] = bbox
            self.bbox_labels[detection['id']] = label

    def update_comment_section(self):
        self.comment_text.delete(1.0, tk.END)
        for detection in self.detections:
            comment = self.detection_comments.get(detection['id'], "")
            if comment:
                self.comment_text.insert(tk.END, f"ID {detection['id']}: {comment}\n")

    def export_modified_dicom(self):
        if self.dicom_image is None:
            messagebox.showerror("Error", "No DICOM image to export.")
            return

        file_path = filedialog.asksaveasfilename(defaultextension=".dcm", filetypes=[("DICOM files", "*.dcm")])
        if file_path:
        # Update the DICOM metadata with bounding box and comment information
        # We will use private tags for this purpose (0019,xx00 to 0019,xxFF are commonly used for private data)
        
        # Clear any existing private tags in the range we plan to use
            for element in self.dicom_image.iterall():
                if (element.tag.group, element.tag.element) >= (0x0019, 0x1000) and (element.tag.group, element.tag.element) <= (0x0019, 0x10FF):
                    del self.dicom_image[element.tag]

        # Add new private tags for bounding boxes and comments
            for detection in self.detections:
                bbox = detection['bbox']
                comment = detection['comment']
                detection_id = detection['id']
                class_name = detection['class']
                confidence = detection['confidence']

                bbox_tag = pydicom.tag.Tag(0x0019, 0x1000 + detection_id)
                comment_tag = pydicom.tag.Tag(0x0019, 0x1080 + detection_id)

                bbox_data = f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]},{class_name},{confidence:.2f}"
                self.dicom_image.add_new(bbox_tag, 'LT', bbox_data)

                if comment:
                    self.dicom_image.add_new(comment_tag, 'LT', comment)

        # Save the updated DICOM
            pydicom.dcmwrite(file_path, self.dicom_image)
            messagebox.showinfo("Success", f"Modified DICOM saved to {file_path}")
    
    def toggle_history_log(self):
        if self.history_frame_visible:
            self.history_frame.pack_forget()
        else:
            self.history_frame.pack(side=tk.RIGHT, fill=tk.Y)
        self.history_frame_visible = not self.history_frame_visible

    def log_history(self, description):
        state = {
            'image': self.processed_image.copy(),
            'description': description,
            'timestamp': datetime.now()
        }
        self.history.append(state)
        self.update_history_listbox()

    def update_history_listbox(self):
        self.history_listbox.delete(0, tk.END)
        for state in self.history:
            description = state['description']
            timestamp = state['timestamp'].strftime('%Y-%m-%d %H:%M:%S')
            self.history_listbox.insert(tk.END, f"{timestamp} - {description}")

    def revert_to_history_item(self, event):
        selected_index = self.history_listbox.curselection()[0]
        selected_state = self.history[selected_index]
        self.processed_image = selected_state['image']
        self.display_image(self.processed_canvas, self.processed_image)

if __name__ == "__main__":
    root = tk.Tk()
    app = MedImagePro(root)
    root.wm_iconbitmap('MedImagePro.ico')
    root.mainloop()     